<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_3_data_processing/part4_custom_search_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Custom Search Agent**

In [1]:
# Install the required packages:
%%capture --no-stderr
%pip install -U  langsmith # check
%pip install --quiet -U  langchain_google_genai langchain_core langchain langgraph   python-dotenv langsmith

In [2]:
%%capture --no-stderr
%pip install ChatGoogleGenerativeAI

In [3]:
# packages
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [4]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ice_breaker"

In [5]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [40]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gemini_api_key, temperature=0)

In [6]:
# llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [7]:
llm.invoke("hello")

AIMessage(content='Hello there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d8adeaee-79ff-42d7-89db-32efcb0db82b-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [16]:
# Restart Runtime and Re-authenticate
from google.colab import auth
auth.authenticate_user()


In [8]:
result = llm.invoke("hi")
print(result)

content='Hi there! How can I help you today?\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-d8b10998-14a9-4019-83d4-58365f33dac9-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}


In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")
# model.invoke("hi")

[Tavily Search](https://python.langchain.com/docs/integrations/tools/tavily_search/)

[TAVILY_API_KEY](https://tavily.com/)


In [9]:
# API keys set up
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

[hwchase17/react](https://smith.langchain.com/hub/hwchase17/react)



**Create React Agent**

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults


def get_profile_url_tavily(name: str):
    """Searches for Linkedin or Twitter Profile Page."""
    search = TavilySearchResults()
    res = search.run(f"{name}")
    return res

In [11]:
from dotenv import load_dotenv

load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (
    create_react_agent,
    AgentExecutor,
)
from langchain import hub
# from tools.tools import get_profile_url_tavily


def lookup(name: str) -> str:
    # llm = ChatGoogleGenerativeAI(
    #     model="gemini-1.5-flash",
    # )
    template = """given the full name {name_of_person} I want you to get it me a link to their Linkedin profile page.
                              Your answer should contain only a URL"""

    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )
    tools_for_agent = [
        Tool(
            name="Crawl Google 4 linkedin profile page",
            func=get_profile_url_tavily,
            description="useful for when you need get the Linkedin Page URL",
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=react_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)

    result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(name_of_person=name)}
    )

    linked_profile_url = result["output"]
    return linked_profile_url


# if __name__ == "__main__":
#     print(lookup(name="Eden Marco Udemy"))

In [12]:
# # Example function call
linkedin_url = lookup(name="Eden Marco Udemy")
print("LinkedIn URL:", linkedin_url)



> Entering new AgentExecutor chain...
Thought: I need to find Eden Marco's LinkedIn profile URL using their full name.  I can use Google to search for their LinkedIn profile.

Action: Crawl Google 4 linkedin profile page

Action Input: Eden Marco Udemy
[{'url': 'https://twitter.com/EdenEmarco177/highlights', 'content': 'Eden Marco. @EdenEmarco177. LLMs. @google. cloud | Best-seller. @udemy. Instructor | Opinions stated here are my own, not those of my company. Software'}, {'url': 'https://www.udemy.com/course/lamaindex/?srsltid=AfmBOoqPVRiUMX68PwtTXJRC3W36evGdI_aBH7OGvCbYvPpOQi09uHSk', 'content': 'Eden Marco | LLM Specialist. Rating: 4.4 out of 54.4 (777) ... Eden Marco as the Instructor is excellent is his communication with the LlamaIndex concepts.'}, {'url': 'https://www.udemy.com/course/langchain/?srsltid=AfmBOorZyaCnPCxa5JWVdLp2ozfvrY__6HpylCCOPlGWb5Jtm4f-eT-O', 'content': "Eden Marco's courses on LangChain and LangGraph are truly exceptional! Eden does a fantastic job of breaki